In [8]:
import open3d as o3d
import os
from os.path import join
import numpy as np
import random

In [324]:
filelist =[ f for f in os.listdir(join(os.getcwd(),"log","pcd")) if f.startswith("original")]
filecnt = len(filelist)
num = random.randint(0,filecnt)
print(filecnt,num)

pcd = o3d.io.read_point_cloud(join(os.getcwd(), "log", "pcd", filelist[num]))
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

48 4


PointCloud with 136330 points.

In [334]:
filelist =[ f for f in os.listdir(join(os.getcwd(),"log","pcd")) if f.startswith("target")]
filecnt = len(filelist)
num = random.randint(0,filecnt-1)
print(filecnt,num)

pcd = o3d.io.read_point_cloud(join(os.getcwd(), "log", "pcd", filelist[num]))
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

112 94


PointCloud with 12229 points.

In [335]:

o3d.visualization.draw_geometries([pcd])

In [340]:
pcd_total_tree = o3d.geometry.KDTreeFlann(pcd)
points_total = np.asarray(pcd.points)
normals_total = np.asarray(pcd.normals)
colors = np.asarray(pcd.colors)
x = np.argwhere(normals_total[:, 2] > 0)
#normals_total[x] = normals_total[x] * -1.0


In [327]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [193]:
points_total

array([[-0.01797023, -0.0200546 , -0.40200001],
       [-0.01972379, -0.02006458, -0.40220001],
       [-0.02081935, -0.02006957, -0.4023    ],
       ...,
       [-0.09361771,  0.03471603, -0.39469999],
       [ 0.00073538,  0.01947152, -0.38139999],
       [-0.09053063,  0.03451813, -0.39245   ]])

In [332]:
normals_total
normal_norm = np.linalg.norm(normals_total)
normal_array = normals_total/normal_norm
point_norm = np.linalg.norm(points_total)
point_array = points_total/point_norm

print(normal_array)
print(normals_total)
print(points_total)
print(point_array)



[[-2.68370689e-03 -8.42401514e-03  9.71994415e-03]
 [-6.02717356e-05 -2.89515150e-03  3.27640376e-03]
 [ 1.83880548e-04 -7.58785477e-03  1.07153494e-02]
 ...
 [-6.69233491e-03 -6.81949412e-03  9.01278500e-03]
 [ 3.69720643e-03  6.36776470e-03  1.08823648e-02]
 [ 9.38862772e-04  2.81080731e-04  8.60356266e-04]]
[[-0.20424893 -0.64112669  0.73975599]
 [-0.0045871  -0.22034135  0.24935733]
 [ 0.0139946  -0.57748902  0.81551331]
 ...
 [-0.50933367 -0.51901138  0.68593621]
 [ 0.28138337  0.4846316   0.82822436]
 [ 0.07145405  0.02139222  0.06547915]]
[[-0.01797023 -0.0200546  -0.40200001]
 [-0.01972379 -0.02006458 -0.40220001]
 [-0.02081935 -0.02006957 -0.4023    ]
 ...
 [-0.09361771  0.03471603 -0.39469999]
 [ 0.00073538  0.01947152 -0.38139999]
 [-0.09053063  0.03451813 -0.39245   ]]
[[-4.23020683e-04 -4.72087037e-04 -9.46311483e-03]
 [-4.64299621e-04 -4.72321924e-04 -9.46782293e-03]
 [-4.90089228e-04 -4.72439346e-04 -9.47017663e-03]
 ...
 [-2.20376889e-03  8.17218391e-04 -9.29127175e-03]

In [347]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.0037,
                                         ransac_n=4,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
radius = 10 / 1000

[prev_target_n, prev_target_idx, _] = pcd_total_tree.search_radius_vector_3d(inlier_cloud.get_center(), radius)
center_cloud = pcd.select_by_index(prev_target_idx)
print(center_cloud)
center_cloud.paint_uniform_color([0, 1.0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud,center_cloud])


Plane equation: 0.70x + 0.47y + 0.55z + 0.19 = 0
PointCloud with 351 points.


In [338]:
#xxx = np.concatenate((points_total,normals_total),axis=1)
xxx = np.concatenate((point_array,normal_array),axis=1)
#xxx = points_total

#pca = PCA(n_components=5).fit(xxx)
#kmeans = KMeans(init=pca.components_,n_clusters=5, random_state=0).fit(xxx)
kmeans = KMeans(n_clusters=2, random_state=0).fit(xxx)

In [294]:
colorset = [ [1.0,0,0],[0,1.0,0],[0,0,1.0],[0.7,0.6,0.9],[0.5,0,0.5],[0,0.5,0.5]]

In [295]:
colors = [ colorset[l] for l in kmeans.labels_ ]

In [296]:
pcd.colors = o3d.utility.Vector3dVector(colors)


In [297]:
o3d.visualization.draw_geometries([pcd])

In [279]:
from sklearn.cluster import DBSCAN

In [283]:
model = DBSCAN(min_samples=2)

y_predict = model.fit_predict(xxx)


In [284]:
set(y_predict)

{0}

In [285]:
xxx

array([[-4.23020683e-04, -4.72087037e-04, -9.46311483e-03,
        -2.68370689e-03, -8.42401514e-03,  9.71994415e-03],
       [-4.64299621e-04, -4.72321924e-04, -9.46782293e-03,
        -6.02717356e-05, -2.89515150e-03,  3.27640376e-03],
       [-4.90089228e-04, -4.72439346e-04, -9.47017663e-03,
         1.83880548e-04, -7.58785477e-03,  1.07153494e-02],
       ...,
       [-2.20376889e-03,  8.17218391e-04, -9.29127175e-03,
        -6.69233491e-03, -6.81949412e-03,  9.01278500e-03],
       [ 1.73108712e-05,  4.58361314e-04, -8.97818857e-03,
         3.69720643e-03,  6.36776470e-03,  1.08823648e-02],
       [-2.13109891e-03,  8.12559841e-04, -9.23830689e-03,
         9.38862772e-04,  2.81080731e-04,  8.60356266e-04]])